In [1]:
!ls -ltr *.json

-rw-r--r-- 1 atif atif 401 Nov 15 21:30 input_1.json
-rw-r--r-- 1 atif atif 479 Nov 17 23:42 nameage.json


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Accumulator_app").getOrCreate()

In [3]:
sc = spark.sparkContext

In [4]:
df = spark.read.option("inferschema","true").json("nameage.json")

In [5]:
df.show(truncate=0)

+---+-------+
|Age|name   |
+---+-------+
|31 |Alex   |
|1  |Anwar  |
|89 |Senthil|
|25 |Azar   |
|55 |Elil   |
|11 |George |
|18 |Mike   |
|21 |Mark   |
|19 |Taj    |
|16 |Thaha  |
|46 |Suresh |
|72 |Hussey |
|35 |gary   |
|45 |Clara  |
|19 |Thara  |
|34 |Radha  |
|3  |Badri  |
|14 |Thaha  |
+---+-------+



## Our requirement : Get count of all the age <18

In [45]:
accumulator_var = sc.accumulator(value=0)

In [46]:
accumulator_var.value

0

In [47]:
from pyspark.accumulators import AccumulatorParam

In [48]:
# Create a custom accumulator for string concatenation
# Contrived example -- see recipe for caveats.
class StringAccumulatorParam(AccumulatorParam):
    def zero(self, initialValue=" "):
        return " "
 
    def addInPlace(self, s1, s2):
        return s1.strip() + " " + s2.strip()

In [49]:
countervalue = sc.accumulator(" ",StringAccumulatorParam())

In [50]:
def validate(row):
    age =row.Age
    if age <18:
        accumulator_var.add(1)
        countervalue.add(str(age))


In [34]:
countervalue = sc.accumulator(" ",StringAccumulatorParam)

In [52]:
accumulator_var.value

5

In [51]:
df.foreach(lambda x:validate(x))